### Import

In [57]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib
import seaborn as sns

# 각 라이브러리 버전 출력
print("Numpy version:", np.__version__)
print("Scikit-learn version:", sklearn.__version__)
print("Pandas version:", pd.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Seaborn version:", sns.__version__)

Numpy version: 1.26.4
Scikit-learn version: 1.5.1
Pandas version: 1.5.3
Matplotlib version: 3.9.2
Seaborn version: 0.13.2


In [58]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [59]:
import sys
import os
import pandas as pd

# 현재 작업 디렉토리 경로를 가져와 shared codes 폴더의 위치를 sys.path에 추가합니다.
# sys.path에 추가된 경로에 있는 py 폴더는 임포트할 수 있다.
current_dir = os.getcwd()
shared_codes_dir = os.path.join(current_dir, '../shared codes')
sys.path.append(shared_codes_dir)


# cover_nan 모듈을 임포트
from cover_nan import missing_value_removal_function

# 원본 train 데이터 로드
train = pd.read_csv("../shared codes/data/train.csv")
test = pd.read_csv("../shared codes/data/test.csv")

# missing_value_removal_function 사용
train = missing_value_removal_function(train)
test = missing_value_removal_function(test)

✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!
✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!


In [60]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [61]:
# 각 열의 결측값 개수 확인
missing_values_count = X.isnull().sum()

# 결측값이 있는 열만 필터링
missing_columns = missing_values_count[missing_values_count > 0]

print("결측값이 있는 열과 개수:")
print(missing_columns)

결측값이 있는 열과 개수:
Series([], dtype: int64)


In [62]:
# '알 수 없음'이 포함된 열 찾기
unknown_columns = []

for col in X.columns:
    if X[col].astype(str).str.contains('알 수 없음').any():
        unknown_columns.append(col)

# 결과 출력
print("⚠️ '알 수 없음' 값을 포함하는 컬럼들:")
print(unknown_columns)


⚠️ '알 수 없음' 값을 포함하는 컬럼들:
['시술 당시 나이', '배란 유도 유형', '난자 기증자 나이', '정자 기증자 나이']


In [63]:
# '알 수 없음'이 포함된 열 찾기
unknown_columns = []

for col in X.columns:
    if X[col].astype(str).str.contains('nan').any():
        unknown_columns.append(col)

# 결과 출력
print("⚠️ 'nan' 값을 포함하는 컬럼들:")
print(unknown_columns)


⚠️ 'nan' 값을 포함하는 컬럼들:
[]


In [64]:
# NaN 값이 존재하는 컬럼 찾기
unknown_columns = X.columns[X.isna().any()].tolist()

unknown_columns

[]

In [65]:
# Categorical(범주형) 칼럼 찾기
categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()

# 결과 출력
print("📌 Categorical(범주형) 칼럼 리스트:")
print(categorical_columns)


📌 Categorical(범주형) 칼럼 리스트:
['ID', '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형', '배란 유도 유형', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이']


In [66]:
X['난자 채취 경과일'].fillna(1, inplace=True)
X['난자 해동 경과일'].fillna(0, inplace=True)
X['난자 혼합 경과일'].fillna(0, inplace=True)
X['배아 이식 경과일'].fillna(0, inplace=True)
X['배아 해동 경과일'].fillna(0, inplace=True)

test['난자 채취 경과일'].fillna(1, inplace=True)
test['난자 해동 경과일'].fillna(0, inplace=True)
test['난자 혼합 경과일'].fillna(0, inplace=True)
test['배아 이식 경과일'].fillna(0, inplace=True)
test['배아 해동 경과일'].fillna(0, inplace=True)


In [67]:
# 병합할 칼럼들
columns_to_merge = [
    '난자 채취 경과일', '난자 해동 경과일', '난자 혼합 경과일',
    '배아 이식 경과일', '배아 해동 경과일'
]

# 새로운 칼럼 생성: 각 경과일의 합
X['총 경과일'] = X[columns_to_merge].sum(axis=1)

# 기존 칼럼 제거
X = X.drop(columns=columns_to_merge)

# 새로운 칼럼 생성: 각 경과일의 합
test['총 경과일'] = test[columns_to_merge].sum(axis=1)

# 기존 칼럼 제거
test = test.drop(columns=columns_to_merge)

In [68]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [69]:
columns_to_drop = [
        "남성 주 불임 원인",
        "남성 부 불임 원인",
        "불임 원인 - 정자 농도",
        "불임 원인 - 정자 면역학적 요인",
        "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태",
        '정자 기증자 나이',
        '배란 유도 유형'
]
X_train_encoded = X_train_encoded.drop(columns = columns_to_drop)    
X_test_encoded = X_test_encoded.drop(columns = columns_to_drop)  

In [70]:
# 각 열의 결측값 개수 확인
missing_values_count = X_train_encoded.isnull().sum()

# 결측값이 있는 열만 필터링
missing_columns = missing_values_count[missing_values_count > 0]

print("결측값이 있는 열과 개수:")
print(missing_columns)


결측값이 있는 열과 개수:
Series([], dtype: int64)


In [71]:
X_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 51 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                256351 non-null  float64
 1   시술 시기 코드          256351 non-null  float64
 2   시술 당시 나이          256351 non-null  float64
 3   시술 유형             256351 non-null  float64
 4   특정 시술 유형          256351 non-null  float64
 5   배란 자극 여부          256351 non-null  int64  
 6   단일 배아 이식 여부       256351 non-null  float64
 7   여성 주 불임 원인        256351 non-null  int64  
 8   여성 부 불임 원인        256351 non-null  int64  
 9   부부 주 불임 원인        256351 non-null  int64  
 10  부부 부 불임 원인        256351 non-null  int64  
 11  불명확 불임 원인         256351 non-null  int64  
 12  불임 원인 - 난관 질환     256351 non-null  int64  
 13  불임 원인 - 남성 요인     256351 non-null  int64  
 14  불임 원인 - 배란 장애     256351 non-null  int64  
 15  불임 원인 - 여성 요인     256351 non-null  int64  
 16  불임 원인 - 자궁경부 문제   25

In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# 데이터 정규화 (X_train_encoded & X_test_encoded)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)  # 동일한 스케일 적용

# DataFrame 변환 (Feature 이름 유지)
feature_names = [f"Feature_{i}" for i in range(X_train_scaled.shape[1])]
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_names)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=feature_names)

# 상관 행렬 계산
correlation_matrix_train = X_train_scaled_df.corr()

# 다중 공선성이 높은 칼럼 찾기 (절대 상관 계수가 0.8 이상)
threshold = 0.8
high_corr_features = set()

for i in range(len(feature_names)):
    for j in range(i + 1, len(feature_names)):
        if abs(correlation_matrix_train.iloc[i, j]) > threshold:
            high_corr_features.add(feature_names[j])  # 공선성이 높은 컬럼 추가

# 다중 공선성이 높은 컬럼 제거
X_train_encoded = X_train_scaled_df.drop(columns=high_corr_features, errors='ignore')
X_test_encoded = X_test_scaled_df.drop(columns=high_corr_features, errors='ignore')


In [73]:
X_train_encoded

,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_10,...,Feature_37,Feature_38,Feature_39,Feature_42,Feature_43,Feature_44,Feature_46,Feature_48,Feature_49,Feature_50
0,-1.732044,1.518507,-0.935575,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.066583,-0.184224,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,-0.140945
1,-1.732031,1.020975,2.655919,0.158613,-0.962962,-1.836376,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.066583,-1.000848,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,-0.140945
2,-1.732017,0.025910,-0.935575,0.158613,0.672585,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.066583,-0.048121,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,-0.773279
3,-1.732004,-0.471622,-0.217277,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.066583,-0.456432,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,-0.140945
4,-1.731990,0.025910,-0.935575,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.066583,-0.184224,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,-0.140945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256346,1.731990,1.020975,-0.935575,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.066583,0.360191,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,1.123721
256347,1.732004,1.020975,0.501022,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.066583,-0.456432,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,-0.140945
256348,1.732017,0.025910,-0.217277,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.066583,0.224087,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,-0.140945
256349,1.732031,1.518507,0.501022,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.066583,-0.728640,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,-1.405612


### Train

In [74]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(random_state=42)

**앙상블 및 Optuna**

In [75]:
import optuna
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

In [76]:
def optimize_xgboost(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5)
    }
    model = XGBClassifier(**params, random_state=42, use_label_encoder=False, eval_metric="logloss")
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")
    return np.mean(scores)

In [77]:
def optimize_lightgbm(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 10, 50),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0)
    }
    model = LGBMClassifier(**params, random_state=42)
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
    return np.mean(scores)

In [78]:
def optimize_random_forest(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_samples_split": trial.suggest_int("min_samples_split", 3, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4)
    }
    model = RandomForestClassifier(**params, random_state=42)
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
    return np.mean(scores)


In [79]:
# Optuna 스터디 실행
print("Optimizing XGBoost...")
xgb_study = optuna.create_study(direction="maximize")
xgb_study.optimize(optimize_xgboost, n_trials=30)
best_xgb_params = xgb_study.best_params

print("Optimizing LightGBM...")
lgbm_study = optuna.create_study(direction="maximize")
lgbm_study.optimize(optimize_lightgbm, n_trials=30)
best_lgbm_params = lgbm_study.best_params

print("Optimizing RandomForest...")
rf_study = optuna.create_study(direction="maximize")
rf_study.optimize(optimize_random_forest, n_trials=30)
best_rf_params = rf_study.best_params

[I 2025-02-14 12:26:58,127] A new study created in memory with name: no-name-8a49abdd-c062-4be5-b66f-81dfba692ad2


Optimizing XGBoost...


c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [12:26:58] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [12:26:58] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [12:26:59] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [12:27:00] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warni

Optimizing LightGBM...
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006998 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 754
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
[LightGBM] [Info] Number of positive: 52983, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007225 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 753
[LightGBM] [Info] Number of data points in the train set: 205081, number of used featur

[I 2025-02-14 12:31:13,618] Trial 0 finished with value: 0.7358774300899643 and parameters: {'n_estimators': 354, 'max_depth': 6, 'learning_rate': 0.04491247522090271, 'num_leaves': 15, 'subsample': 0.6220024892790562}. Best is trial 0 with value: 0.7358774300899643.


...

[I 2025-02-14 12:33:51,210] Trial 28 finished with value: 0.7357657957203667 and parameters: {'n_estimators': 280, 'max_depth': 3, 'learning_rate': 0.034744309382616245, 'num_leaves': 25, 'subsample': 0.7424303753610505}. Best is trial 22 with value: 0.7361370316918461.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-02-14 12:33:56,015] Trial 29 finished with value: 0.7303400386961829 and parameters: {'n_estimators': 388, 'max_depth': 5, 'learning_rate': 0.19743662320805905, 'num_leaves': 10, 'subsample': 0.6327382186731407}. Best is trial 22 with value: 0.7361370316918461.
[I 2025-02-14 12:33:56,016] A new study created in memory with name: no-name-42ff38dd-a847-43cc-a441-5f6c1d5a29b5


Optimizing RandomForest...


[I 2025-02-14 12:34:28,863] Trial 0 finished with value: 0.7285887809565901 and parameters: {'n_estimators': 67, 'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.7285887809565901.
[I 2025-02-14 12:35:31,525] Trial 1 finished with value: 0.732740693401202 and parameters: {'n_estimators': 102, 'max_depth': 11, 'min_samples_split': 10, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.732740693401202.
[I 2025-02-14 12:36:08,013] Trial 2 finished with value: 0.7312329777967715 and parameters: {'n_estimators': 69, 'max_depth': 9, 'min_samples_split': 6, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.732740693401202.
[I 2025-02-14 12:37:05,750] Trial 3 finished with value: 0.71453108104852 and parameters: {'n_estimators': 250, 'max_depth': 3, 'min_samples_split': 4, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.732740693401202.
[I 2025-02-14 12:38:16,925] Trial 4 finished with value: 0.7327381454952893 and parameters: {'n_estima

In [80]:
# 최적화된 모델 생성
xgb_model = XGBClassifier(**best_xgb_params, random_state=42, use_label_encoder=False, eval_metric="logloss")
lgbm_model = LGBMClassifier(**best_lgbm_params, random_state=42)
rf_model = RandomForestClassifier(**best_rf_params, random_state=42)

In [81]:
# XGBoost 모델의 파라미터 확인
print("XGBoost Best Parameters:")
print(xgb_model.get_params())

# LightGBM 모델의 파라미터 확인
print("\nLightGBM Best Parameters:")
print(lgbm_model.get_params())

# RandomForest 모델의 파라미터 확인
print("\nRandomForest Best Parameters:")
print(rf_model.get_params())

XGBoost Best Parameters:
{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.897323073300406, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'logloss', 'feature_types': None, 'gamma': 3.156357001670953, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.04088687836051916, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 3, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 421, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.7742625141464532, 'tree_method': None, 'validate_parameters': None, 'verbosity': None, 'use_label_encoder': 

In [82]:
# Soft Voting 앙상블
ensemble_model = VotingClassifier(
    estimators=[
        ("xgb", xgb_model),
        ("lgbm", lgbm_model),
        ("rf", rf_model)
    ],
    voting="soft"
)

In [83]:
# 전체 데이터로 학습
ensemble_model.fit(X_train_encoded, y)

c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [13:23:11] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008920 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 754
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054568
[LightGBM] [Info] Start training from score -1.054568
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.897323073300406,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None,
                                            gamma=3.156357001670953,
                                            grow_policy=None,
                                            importance_type=None,
                                            interacti...
                                            n_estimators=421, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                             ('lgbm',
                              LGBMClassifier(learning_rate=0.03823442910703775,
                                             max_depth=3, n_estimators=432,
                                             num_leaves=14, random_state=42,
                                             subsample=0.6029360386426401)),
                             ('rf',
                              RandomForestClassifier(max_depth=12,
                                                     min_samples_leaf=3,
                                                     min_samples_split=3,
                                                     n_estimators=266,
                                                     random_state=42))],
                 voting='soft')

### Predict

In [84]:
pred_proba = ensemble_model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [85]:
sample_submission = pd.read_csv('../../sample_submission.csv')
sample_submission['probability'] = pred_proba

In [86]:
sample_submission.to_csv('./Result/baseline_submit.csv', index=False)

In [87]:
pred_proba

array([0.01443299, 0.01492065, 0.15846077, ..., 0.46633487, 0.26196425,
       0.0098689 ])

In [88]:
unique, counts = np.unique(sample_submission['probability'], return_counts=True)
print(unique, counts)

[0.00070509 0.00070716 0.0007123  ... 0.65225085 0.65233884 0.66825987] [1 1 3 ... 1 1 1]
